In [2]:
from netunicorn.base import Experiment, Task, Pipeline
from netunicorn.client.remote import RemoteClient

In [3]:
class DummyTask(Task):
    def run(self):
        import time
        time.sleep(3)
        return 0

pipeline = Pipeline().then(DummyTask()).then(DummyTask())

In [4]:
client = RemoteClient(endpoint="http://127.0.0.1:26611", login="test", password="test")
client.healthcheck()

True

In [6]:
nodes = client.get_nodes()
for node in nodes:
    print(node)

current_nodes = nodes.take(1)

[dockerhost]


In [8]:
experiment = Experiment().map(pipeline, current_nodes)
print(experiment)

<Deployment: Node=dockerhost, executor_id=, prepared=False>


In [9]:
try:
    client.delete_experiment('cool_experiment')
except Exception:
    pass

client.prepare_experiment(experiment, 'cool_experiment')

'cool_experiment'

In [11]:
client.get_experiment_status('cool_experiment')

ExperimentExecutionInformation(status=<ExperimentStatus.READY: 2>, experiment=<Deployment: Node=dockerhost, executor_id=99d83242-588c-420c-a224-e21011e2146e, prepared=True>, execution_result=None)

In [12]:
client.start_execution('cool_experiment')

'cool_experiment'

In [14]:
client.get_experiment_status('cool_experiment')

ExperimentExecutionInformation(status=<ExperimentStatus.FINISHED: 4>, experiment=<Deployment: Node=dockerhost, executor_id=99d83242-588c-420c-a224-e21011e2146e, prepared=True>, execution_result=[DeploymentExecutionResult(node=dockerhost, result=[<Success: defaultdict(<class 'list'>, {'cf2424e1-53a2-4b58-8934-32f8364c0ab2': [<Success: 0>], '8e4674d9-9135-4bf3-a3d0-23807abb299e': [<Success: 0>]})>, ['Parsed configuration: Gateway located on http://gateway:26512\n', 'Current directory: /\n', 'Pipeline loaded from local file, executing.\n', 'Pipeline finished, start reporting results.\n']], error=None)])